# Сравнение open-source солверов и их применение в ритейле

Привет, Habr! На связи отдел аналитики данных X5 Tech.

Сегодня мы поговорим про очень интересный раздел прикладной математики — оптимизацию.

Практически каждый человек ежедневно решает оптимизационные задачи даже не задумываясь об этом.
Например, мы, как правило, хотим минимизировать наши расходы для приобретения необходимых товаров, но чтобы эти товары были максимально полезны.
Полезность здесь у каждого своя — для одних она определяется количеством растительных жиров, для других — наличием привычных товаров, и тд.
Другой пример — мы хотим распределить свой маршрут в отпуске так, чтобы посетить всё, что запланировали с минимальными временными затратами на дорогу, и при этом не забыть почилить на пляже.

Бизнес требования часто сводятся к задаче многокритериальной оптимизации и задаче оптимального управления.
В этих постановках решения, найденные методами оптимизации, чрезвычайно полезны для принятия решений.
Приведём для иллюстрации ещё несколько задач из области ритейла.

**Оптимальное распределение маркетингового бюджета**

Реализовать выделенный бюджет на маркетинговые активности максимально эффективно.
Есть несколько каналов для рекламных акций, выделенный бюджет, цель — максимально выгодно инвестировать бюджет, чтобы суммарный доход со всем коммуникаций был максимален. Также необходимо учесть бизнес ограничения на нагрузку каждого канала + частоту взаимодействия.

**Планирование ассортимента**

Подобрать уровни ассортимента мерчгрупп так, чтобы максимизировать оборот с учетом полочного пространства выделенного на мерчгруппу.

**Закупка товаров**

Задача — распределить бюджет выделенный на закупки для поддержания товарооборота, достаточного уровня сервиса, и при этом достигать определенных финансовых показателей под выделенный бюджет на закупки

**Ценообразование**

Поиск оптимальной конфигурации цен с учетом ценового позиционирования, допустимых ценовых диапазонов для каждого товара и набора бизнес-ограничений. Цены должны максимизировать суммарный доход, а прибыль быть не ниже на заданного уровня.



Для формулировки оптимизационной задачи требуется определенная бизнес-экспертиза, чтобы определить, что хотим максимизировать/минимизировать, какие условия надо соблюдать и т.п.
Это весьма трудоёмкий по времени процесс, из-за чего подробное изучение инструментов решения сформулированных задач зачастую пропускается.
Нам на практике встречались даже реализации в Excel, что для MVP решения является приемлемым подходом, но при дальнейшем масштабировании задачи, необходимо искать более масштабируемое и желательно бесплатное решение.

В этой статье мы рассмотрим несколько открытых солверов, сравним их возможности и скорость работы на модельной задаче ценообразования.


## Общая постановка задачи и её разновидности


Прежде всего рассмотрим постановку задачи в общем виде:

$x$ - вектор размерности $n$

$f(x) \to \min(\max)$ - целевая функция

$g_i(x) \leqslant 0, \ i=1..m$ - ограничения вида неравенства

$h_i(x) = 0, \ j=1..k$ - ограничения вида равенства

$x \in X$ - допустимое множество значений переменных ($ \mathbb{R}, \mathbb{Z}$ и т.п.)

Исходя из практики можно разложить данную постановку на несколько классов в зависимости от вида целевой функции, ограничений и $X$

* __Безусловная оптимизация__ $g_i(x), h_j(x)$ - отсутствуют, $X = \mathbb{R}^n$

* __LP__ (linear programming) - линейное программирование. $f(x), g_i(x), h_j(x)$ - линейные функции, $X = \mathbb{R}_+^n$. 
$\mathbb{R}_+$

* __MILP__ (mixed integer linear programming) - смешанное целочисленное линейное программирование, это задача LP в которой часть переменных являются целочисленными

* __NLP__ (nonlinear programming) - нелинейное программирование, возникает когда хотя бы одна из функций $f(x),\ g_i(x),\ h_j(x)$ нелинейна

* __MINLP__ (mixed integer nonlinear programming) - смешанное целочисленное нелинейное программирование, возникает как и в MILP, когда часть переменных принимает целочисленные значения


__NLP__ в свою очередь можно подробить еще на множество разных классов в зависимости от вида нелинейности и выпуклости.


## Обзор пакетов python

В указанных выше задачах возникает так называемая условная оптимизация, здесь предлагается рассмотреть полезные open-source пакеты и солверы для решения задач
условной оптимизации в зависимости от ее типа, которые часто можно встретить в практических задачах.


### Открытые библиотеки, предоставляющие интерфейс для решения оптимизационных задач

__Scipy__ - библиотека, которая содержит большой набор функций для научных вычислений, в том числе имеет инструменты для решения оптимизационных задач, находящиеся в модуле scipy.optimize.
В модуле находятся методы для решения задач линейного программирования, нелинейного программирования (как условная, так и безусловная).
В документации [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) можно найти,
что условную оптимизацию с нелинейностью поддерживают __cobyla__, __slsqp__, __trust-constr__, также там можно найти краткое описание методов и ссылки на статьи, [статья на хабре](https://habr.com/ru/company/ods/blog/448054/)

[__Pyomo__](http://www.pyomo.org/) - пакет, который содержит ряд инструментов для формулирования, решения и анализа оптимизационных моделей.
Главная особенность — это удобный интерфейс для структурированного формулирования оптимизационной задачи и поддержка большого количества солверов, в том числе коммерческих.
По сути pyomo занимается “перевариванием” сформулированной модели в формат, понятный для запускаемого солвера, потом забирает его и выплевывает в интерфейс python.
Является частью проекта [COIN-OR](https://www.coin-or.org/), который содержит также ряд солверов.
Среди них можно выделить [Ipopt](https://github.com/coin-or/Ipopt), [Cbc](https://github.com/coin-or/Cbc).
Ipopt позволяет находить локальные оптимумы в задаче NLP с помощью прямо-двойственного метода внутренней точки, подробнее в оригинальной [статье](http://www.optimization-online.org/DB_HTML/2004/03/836.html).
В Cbc реализован метод решения задачи MILP, основанный на алгоритме, сочетающем в себе метод ветвей и границ и секущих плоскостей [wiki](https://en.wikipedia.org/wiki/Branch_and_cut).
Также для LP имеется поддержка пакета [glpk](https://en.wikipedia.org/wiki/GNU_Linear_Programming_Kit).
И напоследок отметим еще один солвер [bonmin](https://github.com/coin-or/Bonmin), который построен поверх __Cbc__ и __Ipopt__ - сочетание двух солверов, позволяющее браться за задачи __MINLP__.

[__Cvxpy__](https://www.cvxpy.org/index.html) - данный пакет специально заточен для решения задач выпуклой оптимизации (convex optimization).
После того как задача сформулирована, перед решением проверяется выпуклость и аффинность целевой функции и ограничений с помощью правил [DCP](https://www.cvxpy.org/tutorial/dcp/index.html#:~:text=Disciplined%20convex%20programming%20(DCP)%20is,they%20are%20applied%20by%20CVXPY)
(disciplined convex programming), после проверки задача преобразуется в стандартную форму и передается квадратичному или коническому солверу.
Полный список солверов и решаемых задач можно найти [здесь](https://www.cvxpy.org/tutorial/advanced/index.html)

Итого можно сформировать таблицу с солверами, библиотеками и видами задач, которые позволяют решать солверы:

| Солвер(метод) | Пакеты в python | NLP | LP | MILP | MINLP |
|:-------------:|:---------------:|:---:|:--:|:----:|:-----:|
|     cobyla    |      scipy      |  y  |  n |   n  |    n  |
|     slsqp     |      scipy      |  y  |  n |   n  |    n  |
|  trust-constr |      scipy      |  y  |  n |   n  |    n  |
|     ipopt     |      pyomo      |  y  |  y |   n  |    n  |
|      glpk     |   pyomo, cvxpy  |  n  |  y |   y  |    n  |
|      cbc      |   pyomo, cvxpy  |  n  |  y |   y  |    n  |
|      bonmin      |   pyomo  |  y  |  y |   y  |    y  |


## Модельная задача оптимизации в ценообразовании

Как уже было сказано выше, одной из основных целей ценообразования может быть формирование таких цен, которые:

* Соответствуют некоторому набору правил, бизнес-логике и т.п.

* Позволяют достигать определенных целей, например, повышение выручки, трафика, без просадки в других показателях.

Первый пункт по сути задает нам диапазон в котором производится поиск новой цены, возможные соотношения между ценами на товары, соотношения цен с ценами конкурентов, условия на (не)изменение цены.

Второй задает нам целевой функционал для задачи оптимизации и набор ограничений, которые контролируют остальные показатели.

### Обозначения и сокращения

Введем обозначения:

$n$ - количество товаров,

$C_i$ - себестоимость товара $i$,

$R$ - выручка,

$M$ - маржа,

$Q$ - спрос по новой цене $P$,

$Q_0$ - спрос по текущей цене $P_{0}$,

$E$ - коэффициент эластичности (отклик).

$\gamma$ - набор индексов товаров, находящихся в одной линейке, $\gamma_{j}$ - $j-$ая линейка, в которой $m_j$ товаров, $\gamma_{j, k}$ - $k$-ый товар в $j$-ой линейке.


Для удобства введем переменную $x = \frac{P}{P_0}$.


### Модель

__Рассмотрим следующую постановку задачи__:

Необходимо поднять выручку, при условии, что маржа не должна просесть, т.е. оставаться на уровне не ниже текущей в абсолютном выражении.

При этом необходимо соблюдать следующие ценовые границы:
$\pm 10$% от текущей цены, $\pm 15$% от среднерыночной цены, если диапазоны накладываются,
то финальный диапазон — пересечение, в противном случае отклонение от рынка не рассматривается, остается только отклонение от текущей цены.
Также необходимо обеспечивать равенство цен товаров, находящихся в одной линейке товаров (товары которые отличаются только вкусом).
И в конце не забыть, что цена должна быть формата x.99

Чтобы начать решать задачу, требуется знать отклик изменения спроса на изменение цены.
Здесь мы не будем отдельно останавливаться на том как оценивать этот отклик, будем считать что он задан в следующем виде:

\begin{equation}
\tag{1.1}
Q(P) = Q_{0} \exp\bigg(E \cdot \bigg(\frac{P}{P_0} - 1\bigg)\bigg).
\end{equation}

С учётом обозначений, функцию спроса можно переписать в следующем виде:

\begin{equation}
\tag{1.2}
Q(x) = Q_0 \exp(E \cdot (x-1))
\end{equation}

Можно сформулировать поставленную задачу как задачу __NLP__ или __LP__, рассмотрим оба варианта.


### __NLP__ постановка задачи


$$
x \in \mathbb{R} ^ n
$$

\begin{equation}
\tag{2.1}
R(x) = \sum_{i=1}^{n} P_i \cdot Q_i(x_i) = \sum_{i=1}^{n} P_{0, i} \cdot Q_{0, i} \exp{(E_i\cdot(x_i-1))} \to \max_{x}
\end{equation}

\begin{equation}
\tag{2.2}
x_i \in [x_{l, i}, x_{u, i}], \ i=1..n
\end{equation}

\begin{equation}
\tag{2.3}
M(x) = \sum_{i=1}^{n} (P_i - C_i) \cdot Q_i(x_i) = \sum_{i=1}^{n} (P_{0, i} - C_i) \cdot Q_{0, i} \exp{(E_i\cdot(x_i-1))} \geqslant M_0
\end{equation}

\begin{equation}
\tag{2.4.1}
x_{\gamma_{j, 1}} = x_{\gamma_{j, 2}} = ... = x_{\gamma_{j, m_j}}, \ j=1..k
\end{equation}

Ограничение (2.4.1) также можно записать в матричной форме:

\begin{equation}
\tag{2.4.2}
A \cdot x = 0, 
\end{equation}
Все элементы матрицы нулевые, кроме
\begin{equation}
\ A(\gamma_{j,1}, \gamma_{j, 1}) = -A(\gamma_{j, 1}, \gamma_{j, 2}) = 1,\ ..\ , A(\gamma_{j,m_j-1}, \gamma_{j, m_j-1}) = -A(\gamma_{j, m_j-1}, \gamma_{j, m_j}) = 1, \ j=1..k
\end{equation}

(2.1) - это целевой функционал, (2.2) - ценовой диапазон, в котором ищется цена для каждого товара, (2.3) - ограничение на маржу снизу, (2.4.1), (2.4.2) - описывает равенство цен в линейках,
 здесь используются $x$, так как подразумевается, что текущая цена является одинаковой во всей линейке.
В данной постановке $M_0$ - это текущее значение маржи:

\begin{equation}
M_0 = \sum_{i=1}^{n} (P_i - C_i) \cdot Q_{0, i} 
\end{equation}


Цена, которая будет пересчитана по (1.2) является непрерывной из диапазона (2.2), поэтому после необходимо округлить эти цены.
Конечно, округление несколько поломает "оптимальность"


### __MILP__ постановка задачи


Преимуществом данного подхода является то, что цены можно сразу искать по сетке, которые удовлетворяют ограничению (2.2) и правилам округления.
Например, если текущая цена 99.99, то сетка цен может быть такой $\hat{P}$ = [89.99; 90.99; ... 108.99; 109.99], всего 21 значение,
новую цену можно закодировать бинарной маской длиной 21 с одной единичкой, этого можно добиться введя переменную - бинарный вектор длины 21 $\hat{x}$,
у которого сумма всех элементов равна 1: $sum(\hat{x}) = 1$, тогда точно будет выбрана одна цена.
Для каждой цены необходимо получить сетку значений продаж $\hat{Q}$, при этом в общем случае нет необходимости в наличии аналитической функции спроса,
$\hat{Q}$ можно брать хоть из бустинга или т.п.
Что касается линеек, можно было бы ввести дополнительные ограничения на равенство $\hat{x}$ по аналогии с (2.4.1),
но намного проще и дешевле схлопнуть $\hat{Q}$ по линейкам, так как цены внутри них будут искаться по одной и той же сетке и поиск, соответственно, производить по линейкам.

Итого, чтобы сформулировать задачу, нужно:
* сформировать сетку цен размера $g_i$ для каждого товара $\hat{P}_i$, которые удовлетворяют граничным условиям и правилам округления
* задать бинарный вектор $\hat{x}_i$ 
* посчитать уровень спроса $\hat{Q}_i$ для сетки цен $\hat{P}_i$
* схлопнуть $\hat{Q}_i$ по линейкам, взяв сумму

Обозначим за $n'$ - количество линеек.

Таким образом, получим такую постановку:


\begin{equation}
\tag{3.1}
\hat{x}_i \in \{0, 1\}^{g_i},\ i=1..n'
\end{equation}

\begin{equation}
\tag{3.2}
R(\hat{x}) = \sum_{i=1}^{n'} \sum_{j=1}^{g_i} \hat{P}_{i, j} \cdot \hat{Q}_{i, j} \cdot \hat{x}_{i, j}
\end{equation}

\begin{equation}
\tag{3.3}
\sum_{j=1}^{g_i} \hat{x}_{i, j} = 1,\ i=1..n'
\end{equation}

\begin{equation}
\tag{3.4}
M(\hat{x}) = \sum_{i=1}^{n'} \sum_{j=1}^{g_i} (\hat{P}_{i, j} - C_i) \cdot \hat{Q}_{i, j} \hat{x}_{i, j} \geqslant M_0
\end{equation}


## Описание подхода к решению оптимизационной задачи

Решение задач будем производить с помощью библиотек и солверов, указанных выше.
Slsqp, cobyla, trust-constr из scipy, ipopt из pyomo - будут использованы для решения задачи с непрерывными переменными (2.\*).
Cobyla не поддерживает ограничения типа равенства, коими здесь являются (2.4.1), но это не помешает нам, так как это ограничение просто равенство переменных,
а значит внутри группы можно задать всего одну переменную — она будет общей для линейки.
То же самое можно сделать и для всех остальных NLP солверов из scipy, а вот в pyomo можно явно задать равенство переменных, так в процессе подготовки он сам сделает замену на одну переменную.
Что касается cvxpy, то несложно увидеть, что исходный функционал не является выпуклым.
Более того, в общем случае, исходя из логических соображений можно прикинуть, что при увеличении цены спрос и вместе с ним выручка будут асимптотически падать до нуля,
а при уменьшении цены ближе к нулю выручка будет стремиться к какому-то большому числу.
То есть функция, описывающая выручку от цены, будет как минимум содержать выпуклость вниз(вогнутость), что при условии максимизации функции не удовлетворяет условиям выпуклого программирования.
Задача (3.\*) - будет решаться с помощью солверов glpk, cbc из пакетов pyomo, cvxpy и ecos из cvxpy.

## Пайплайн

Пайплайн решения задачи можно разбить на три этапа:
* 1 - препроцессинг, в который входит подготовка данных, формирование ценовых правил.
* 2 - непосредственно сам шаг с решением оптимизационной задачи.
* 3 - постпроцессинг — анализ полученного решения в случае успеха.

Третий этап необходим, т.к. может сложится, что задачу на шаге 2 не удалось решить.
Например, из-за пустого допустимого множества, т.е. множества, в котором переменные удовлетворяют всем ограничениям.
В таких случаях можно просто подсветить, что задача не имеет решения или разработать схему исключения/ослабления ограничений и вернуться к шагу 2 с обновленными условиями.
Обработка таких случаев это тоже отдельная и не всегда тривиальная задача, в этой статье касаться не будем.

### 1. Препроцессинг

Начнем с шага 1.
Для модельной задачи ценообразования был реализован генератор данных, который по заданным параметрам формирует все необходимые данные для двух постановок задач, код генератора описан [здесь](https://github.com/mbudylin/OptimizersArticle/blob/main/data_generator/data_generator.py).
На входе — количество линеек, ценовые диапазоны, заданные словарем, размер сетки для задачи (3.\*) и seed для фиксации генератора случайных чисел.
На выходе получаем данные для задачи NLP, MILP и словарь, содержащий список товаров для каждой линейки.
При генерации данных опираемся на количество линеек, так как выше было показано, что поиск цен происходит внутри линейки, т.е. необходима одна переменная(или вектор в LP) на одну линейку.
Размер сетки для поиска в MILP постановке возьмем 21 = текущая цена и по 10 цен в каждую сторону в рамках диапазона +- 10% с дальнейшей отсечкой по границам (2.2) с дальнейшим округлением.
В результате округления можем слегка выйти за основной диапазон — будем считать это допустимым.
Для удобства выполнения матричных операций, в случае, если финальный размер сетки для товара меньше заданной, то дополняем сетку нулями справа.

```python
# пример кода для запуска генератора данных
from data_generator.data_generator import generate_data

N = 5
bounds_params = {
    'main_bounds': {
        'lower': 0.9, 'upper': 1.1
    },
    'market_bounds': {
        'lower': 0.85, 'upper': 1.15
    }
}
grid_size = 21
seed = 10
data = generate_data(N, bounds_params, grid_size, seed)
```

<details>
<summary> |> Пример данных для NLP постановки </summary>
<div><pre><code class="python">
# plu_line - код продуктовой линейки
# plu - код товара
# plu_idx - индекс товара
# P - текущая цена
# Q - текущие продажи в штуках
# E - эластичность
# PC - цена конкурента/рыночная цена
# C - себестоимость товара
# x_lower - нижняя граница x для диапазона поиска цены
# x_upper - верхняя граница x для диапазона поиска цены
# x_init - начальное значение x для старта оптимизатора
# fixed - метка, необходимо ли фиксировать x_init цену на товар, по умолчанию всегда 0
data['data_nlp']

<img src="images/data_nlp_sample.png" width="550" align="center"/>
</code></pre><p></p></div>
</details>



<details>
<summary> |> Пример данных для MILP постановки </summary>
<div><pre><code class="python">
Пример данных для MILP постановки
# plu_line - код продуктовой линейки
# Ps - сетка цен для поиска
# Qs - сетка продаж для кажой цены из Ps
# xs - сетка индексов
# grid_size - размер сетки
# P_idx - индекс текущей цены в сетке. Если значение -1, то текщая цена не попала в сетку
data['nlp']

<img src="./images/data_milp_sample.png" width="1000" align="c"/>
</code></pre><p></p></div>
</details>


### 2. Расчётная часть

На шаге 2 будет решаться сама оптимизационная задача.
Выглядит удобным описать оптимизационную модель в виде класса, который будет содержать шаги, необходимые для постановки задачи.
Шаги в классе: задание переменных и границ, целевой функции, ограничений и метод, который будет решать оптимизационную задачу.
Структуру в минимальном виде для выше поставленных задач можно описать следующим образом:

```python
class OptimzitaionModel:
    def __init__(self, data):
        """
        Инициализация оптимизационной модели входными данными и объявление объектов для формирования модели
        """
        pass
    
    def init_variables(self):
        """
        Инициализация переменных для оптимизации и задание ценовых границ
        """
        pass
    
    def init_objective(self):
        """
        Формирование целевой функции оптимизации
        """
        pass
    
    def add_con_mrg(self, m_min):
        """
        Задание ограничения на маржу снизу
        """
        pass
    
    def solve(self):
        """
        Метод, который производит поиск оптимального решения, возвращает статус 
        решения оптимизационной задачи, само решение и объект модели из библиотеки, если есть
        """
        pass
    ...
```

Таким образом, можно организовать единый интерфейс для решения различными пакетами.
Реализация методов для каждой библиотеки будет отличаться, что будет описано дальше.

Запуск оптимизационной задачи теперь можно описать в виде функции, в которую поступают данные,
модель для решения задачи, параметры ограничений — словарь и дополнительные опции для солвера [ссылка на github](https://github.com/mbudylin/OptimizersArticle/blob/main/optimizers/optimization.py).


```python
def pricing_optimization(data, opt_model, opt_params, solver, solver_option={}):
    """
    Запуск расчета оптимальных цен с помощью указанного класса оптимизатора и параметров
    :param data: входные данные для оптимизации
    :param opt_model: класс модели оптимизатора
    :param opt_params: параметры оптимизации
    :param solver: солвер для оптимизации
    :param solver_option: параметры солвера
    :return: словарь, возвращаемый моделью оптимизации
    """
    alpha = opt_params['alpha']
    model = opt_model(data, alpha)

    model.init_variables()
    model.init_objective()

    if opt_params.get('con_mrg') is not None:
        model.add_con_mrg(opt_params['con_mrg'])

    result = model.solve(solver=solver, options=solver_option)
    result['model_class'] = model
    
    return result
```

Как можно видеть через opt_params можно управлять ограничениями, включать их по необходимости и задавать границы для них, для наших задач он будет выглядеть так:

```python
opt_params = {
    'alpha': 0.0,
    'con_mrg': M_cur,
}
```


### Реализации классов для оптимизационных задач

Описание классов здесь займет достаточно много места, поэтому они будут спрятаны под спойлер, можно под него провалиться и просмотреть — реализация сопровождена комментариями.
Также можно найти все по ссылке на [github](https://github.com/mbudylin/OptimizersArticle/blob/main/optimizers/optimizers.py)


<details class="spoiler">
<summary>
|> Импорты 
</summary>
<div class="spoiler__content"><pre><code class="python">
import numpy as np
import pandas as pd
from scipy.optimize import NonlinearConstraint, LinearConstraint, minimize
import cvxpy as cp
import pyomo.environ as pyo
</code></pre><p></p></div></details>

<details class="spoiler">
<summary>
|> Описание класа для NLP постановки через Scipy
</summary>
<div class="spoiler__content"><pre><code class="python">
class ScipyNlpOptimizationModel:
    """
    Класс, который создаёт оптимизационную модель на базе библиотеки scipy
    """

    def __init__(self, data: pd.DataFrame, alpha: int):
        if (alpha < 0.0) | (alpha > 1.0):
            raise ValueError('alpha должен быть между 0 и 1')
        self.alpha = float(alpha)
        self.data = data['data_nlp'].copy()
        self.plu_idx_in_line = data['plu_idx_in_line'].copy()
        self.N = len(self.data['plu'])
        self.plu = self.data['plu'].values
        self.P = self.data['P'].values
        self.Q = self.data['Q'].values
        self.E = self.data['E'].values
        self.PC = self.data['PC'].values
        self.C = self.data['C'].values
        # границы для индексов
        self.x_cur = self.data['x_cur'].values
        self.x_lower = self.data['x_lower'].values
        self.x_upper = self.data['x_upper'].values
        self.x_init = self.data['x_init'].values
        self.fixed = self.data['fixed'].values
        # Задаём объект для модели scipy
        self.obj = None
        self.jac = None
        self.bounds = None
        self.x0 = None
        self.constraints = []

        self.k = 0.1 * sum(self.P * self.Q)

    def _el(self, E, x):
        return np.exp(E * (x - 1.0))

    def init_variables(self):
        """
        Инициализация переменных в модели
        """
        self.bounds = list(zip(self.x_lower, self.x_upper))
        self.x0 = 0.5 * (self.x_lower + self.x_upper)
        A = np.eye(self.N, self.N)
        constr_bounds = LinearConstraint(A, self.x_lower, self.x_upper)
        self.constraints.append(constr_bounds)

    def init_objective(self):
        """
        Инициализация целевой функции(выручка или фронт-маржа)
        """
        def objective(x):
            f = -sum(
                (self.P * x - self.alpha * self.C) * self.Q * self._el(self.E, x)
            )
            return f / self.k

        self.obj = objective

    def add_con_rev(self, s_min, s_max=np.inf):
        """
        Добавление в модель ограничения на выручку
        """

        def con_rev(x):
            s = sum(self.P * x * self.Q * self._el(self.E, x))
            return s

        constr = NonlinearConstraint(con_rev, s_min, s_max)
        self.constraints.append(constr)

    def add_con_mrg(self, m_min, m_max=np.inf):
        """
        Добавление в модель ограничения на маржу
        """

        def con_mrg(x):
            m = sum((self.P * x - self.C) * self.Q * self._el(self.E, x))
            return m

        constr = NonlinearConstraint(con_mrg, m_min, m_max)
        self.constraints.append(constr)

    def add_con_equal(self):
        """
        Добавление ограничения на равенство цен внутри группы
        """
        n_con = sum(len(plu_idx) for plu_idx in self.plu_idx_in_line.values()) - len(self.plu_idx_in_line)

        if len(self.plu_idx_in_line) == 0:
            return

        A = np.zeros((n_con, self.N))
        i_con = 0
        for plu_line, plu_idxes in self.plu_idx_in_line.items():
            for plu_idx1, plu_idx2 in zip(plu_idxes[:-1], plu_idxes[1:]):
                A[i_con, plu_idx1] = 1.
                A[i_con, plu_idx2] = -1.
                i_con += 1
        constr = LinearConstraint(A, 0.0, 0.0)
        self.constraints.append(constr)

    def add_con_chg(self, chg_max=None):
        """
        Добавление в модель ограничения на 'разброс' цен
        """
        chg_max2 = chg_max ** 2

        def con_chg(x):
            r = sum((x - 1.0) ** 2) / self.N
            return r

        constr = NonlinearConstraint(con_chg, -np.inf, chg_max2)
        self.constraints.append(constr)

    def solve(self, solver='slsqp', options={}):
        """
        Метод, запускающий решение поставленной оптимизационной задачи
        """
        result = minimize(self.obj, self.x0, method=solver,
                          constraints=self.constraints, options=options)
        self.data['x_opt'] = result['x']
        self.data['P_opt'] = result['x'] * self.data['P']
        self.data['Q_opt'] = self.Q * self._el(self.E, result['x'])

        return {
            'message': str(result['message']),
            'status': str(result['status']),
            'model': result,
            'data': self.data
        }

</code></pre><p></p></div></details>

<details class="spoiler">
<summary>
|> Описание класа для NLP постановки чеерз Pyomo
</summary>
<div class="spoiler__content"><pre><code class="python">
class PyomoNlpOptimizationModel:
    """
    Класс, который создаёт оптимизационную модель на базе библиотеки pyomo
    """

    def __init__(self, data: pd.DataFrame, alpha: int):
        if (alpha < 0.0) | (alpha > 1.0):
            raise ValueError('alpha должен быть между 0 и 1')
        self.alpha = float(alpha)
        self.data = data['data_nlp'].copy()
        self.plu_idx_in_line = data['plu_idx_in_line'].copy()
        self.N = len(self.data['plu'])
        self.plu = self.data['plu'].to_list()
        self.P = self.data['P'].to_list()
        self.Q = self.data['Q'].to_list()
        self.E = self.data['E'].to_list()
        self.PC = self.data['PC'].to_list()
        self.C = self.data['C'].to_list()
        # границы для индексов
        self.x_lower = self.data['x_lower'].to_list()
        self.x_upper = self.data['x_upper'].to_list()
        self.x_init = self.data['x_init'].to_list()
        self.fixed = self.data['fixed'].to_list()
        self.model = pyo.ConcreteModel()

    def _el(self, i):
        # вспомогательная функция для пересчета спроса при изменении цены
        return pyo.exp(self.E[i] * (self.model.x[i] - 1.0))

    def init_variables(self):
        def bounds_fun(model, i):
            return self.x_lower[i], self.x_upper[i]

        def init_fun(model, i):
            return self.x_init[i]

        self.model.x = pyo.Var(range(self.N), domain=pyo.Reals, bounds=bounds_fun, initialize=init_fun)
        for i, fix in enumerate(self.fixed):
            if fix:
                self.model.x[i].fixed = True

    def init_objective(self):
        """
        Инициализация целевой функции(выручка или фронт-маржа)
        """
        objective = sum(
            (self.P[i] * self.model.x[i] - self.alpha * self.C[i]) *
            self.Q[i] * self._el(i) for i in range(self.N)
        )
        self.model.obj = pyo.Objective(expr=objective, sense=pyo.maximize)

    def add_con_rev(self, s_min, s_max=None):
        """
        Добавление в модель ограничения на выручку
        """

        def con_rev(model):
            r = sum(
                self.P[i] * model.x[i] * self.Q[i] * self._el(model, i)
                for i in range(self.N)
            )
            return s_min, r, s_max

        self.model.con_rev = pyo.Constraint(rule=con_rev)

    def add_con_mrg(self, m_min, m_max=None):
        """
        Добавление в модель ограничения на маржу
        """

        def con_mrg(model):
            r = sum(
                (self.P[i] * model.x[i] - self.C[i]) * self.Q[i] * self._el(i)
                for i in range(self.N)
            )
            return m_min, r, m_max

        self.model.con_mrg = pyo.Constraint(rule=con_mrg)

    def add_con_equal(self):
        """
        Добавление ограничения на равенство цен внутри линейки
        """
        if len(self.plu_idx_in_line) == 0:
            return

        self.model.con_equal = pyo.Constraint(pyo.Any)
        # название ограничения = plu_line
        for con_idx, idxes in self.plu_idx_in_line.items():
            for i in range(1, len(idxes)):
                con_name = str(con_idx) + '_' + str(i)
                self.model.con_equal[con_name] = (self.model.x[idxes[i]] - self.model.x[idxes[i - 1]]) == 0

    def add_con_chg(self, chg_max=None):
        """
        Добавление в модель ограничения на 'разброс' цен относительно текущей
        """
        chg_max2 = chg_max ** 2

        def con_chg(model):
            r = sum(
                ((self.model.x[i]) - 1) ** 2
                for i in range(self.N)
            ) / self.N
            return None, r, chg_max2

        self.model.con_chg = pyo.Constraint(rule=con_chg)

    def add_con_chg_cnt(self, thr_l=0.98, thr_u=1.02, nmax=10000):
        """
        Добавление в модель ограничения на изменение цен
        """
        self.model.ind_l = pyo.Var(range(self.N), domain=pyo.Binary, initialize=0)
        self.model.ind_r = pyo.Var(range(self.N), domain=pyo.Binary, initialize=0)
        self.model.ind_m = pyo.Var(range(self.N), domain=pyo.Binary, initialize=1)

        self.model.x_interval = pyo.Constraint(pyo.Any)
        self.model.con_ind = pyo.Constraint(pyo.Any)

        for i in range(self.N):
            self.model.x_interval['l' + str(i)] = self.model.x[i] <= thr_l + 10. * (1 - self.model.ind_l[i])
            self.model.x_interval['r' + str(i)] = self.model.x[i] >= thr_u - 10. * (1 - self.model.ind_r[i])
            self.model.x_interval['ml' + str(i)] = self.model.x[i] <= 1. + 10. * (1 - self.model.ind_m[i])
            self.model.x_interval['mr' + str(i)] = self.model.x[i] >= 1. - 10. * (1 - self.model.ind_m[i])
            self.model.con_ind[i] = (self.model.ind_l[i] + self.model.ind_m[i] + self.model.ind_r[i]) == 1

        self.model.con_max_chg = pyo.Constraint(expr=sum(
            self.model.ind_m[i]
            for i in range(self.N)
        ) >= self.N - nmax)

    def solve(self, solver='ipopt', options={}):
        """
        Метод, запускающий решение поставленной оптимизационной задачи
        """
        solver = pyo.SolverFactory(solver, tee=False)
        for option_name, option_value in options.items():
            solver.options[option_name] = option_value

        result = solver.solve(self.model)

        self.data['x_opt'] = [self.model.x[i].value for i in self.model.x]
        self.data['P_opt'] = self.data['x_opt'] * self.data['P']
        self.data['Q_opt'] = [self.Q[i] * pyo.value(self._el(i)) for i in self.model.x]

        return {
            'message': str(result.solver.termination_condition),
            'status': str(result.solver.status),
            'model': self.model,
            'data': self.data
        }

</code></pre><p></p></div></details>

<details class="spoiler">
<summary>
|> Описание класа для MILP постановки чеерз Pyomo
</summary>
<div class="spoiler__content"><pre><code class="python">
class PyomoLpOptimizationModel:
    """
    Класс, который создаёт оптимизационную модель на базе библиотеки pyomo
    """

    def __init__(self, data: pd.DataFrame, alpha: int):
        if (alpha < 0.0) | (alpha > 1.0):
            raise ValueError('alpha должен быть между 0 и 1')
        self.alpha = float(alpha)
        self.data = data['data_milp'].copy()
        self.N = len(self.data['plu_line'])
        self.grid_size = self.data['grid_size'].to_list()
        self.g_max = max(self.grid_size)
        self.plu_line = self.data['plu_line'].to_list()
        self.n_plu = self.data['n_plu'].to_list()
        self.P = self.data['P'].to_list()
        self.P_idx = self.data['P_idx'].to_list()
        self.PC = self.data['PC'].to_list()
        self.Ps = self.data['Ps'].to_list()
        self.Qs = self.data['Qs'].to_list()
        self.C = self.data['C'].to_list()
        # границы для индексов
        self.xs = self.data['xs'].to_list()
        self.fixed = self.data['fixed'].to_list()
        # Задаём объект модели pyomo
        self.model = pyo.ConcreteModel()

    def init_variables(self):
        """
        Инициализация переменных в модели
        """

        # задаем бинарную метку для цены
        def init_fun(model, i, j):
            return 1 if self.P_idx[i] == j else 0

        self.model.x = pyo.Var(range(self.N), range(self.g_max), initialize=init_fun, domain=pyo.Binary)
        # одна единичка для каждого товара
        self.model.con_any_price = pyo.Constraint(pyo.Any)
        for i in range(self.N):
            self.model.con_any_price[i] = sum(self.model.x[i, j] for j in range(self.grid_size[i])) == 1

        for i, fix in enumerate(self.fixed):
            if fix:
                self.model.x[i, self.P_idx] = 1
                self.model.x[i, :].fixed = True

    def init_objective(self):
        """
        Инициализация целевой функции(выручка или фронт-маржа)
        """
        def objective(model):
            return sum(
                (self.Ps[i][j] * model.x[i, j] - self.alpha * self.C[i]) * self.Qs[i][j]
                for i in range(self.N)
                for j in range(self.grid_size[i])
            )

        self.model.obj = pyo.Objective(rule=objective, sense=pyo.maximize)

    def add_con_rev(self, s_min, s_max=None):
        """
        Добавление в модель ограничения на РТО
        """
        def con_rev(model):
            r = sum(
                self.Ps[i][j] * model.x[i, j] * self.Qs[i][j]
                for i in range(self.N)
                for j in range(self.grid_size[i])
            )
            return s_min, r, s_max

        self.model.con_rev = pyo.Constraint(rule=con_rev)

    def add_con_mrg(self, m_min, m_max=None):
        """
        Добавление в модель ограничения на маржу
        """
        def con_mrg(model):
            r = sum(
                model.x[i, j] * (self.Ps[i][j] - self.C[i]) * self.Qs[i][j]
                for i in range(self.N)
                for j in range(self.grid_size[i])
            )
            return m_min, r, m_max

        self.model.con_mrg = pyo.Constraint(rule=con_mrg)

    def add_con_equal(self):
        pass

    def add_con_chg_cnt(self, nmax=10000):
        """
        Добавление ограничения на количество изменяемых цен
        """
        con_expr = sum(self.model.x[i, self.P_idx[i]] * self.n_plu[i]
                       for i in range(self.N) if self.P_idx[i] > 0) >= sum(self.n_plu) - nmax
        self.model.con_chg_cnt = pyo.Constraint(expr=con_expr)

    def solve(self, solver='cbc', options={}):
        """
        Метод, запускающий решение поставленной оптимизационной задачи
        """
        solver = pyo.SolverFactory(solver)
        for option_name, option_value in options.items():
            solver.options[option_name] = option_value
        result = solver.solve(self.model)
        x_sol = [[self.model.x[i, j].value for j in range(self.grid_size[i])] for i in range(self.N)]
        x_opt_idx = [np.argmax(x_sol[i]) for i in range(self.N)]
        x_opt = [self.xs[i][x_opt_idx[i]] for i in range(self.N)]
        P_opt = [self.Ps[i][x_opt_idx[i]] for i in range(self.N)]
        Q_opt = [self.Qs[i][x_opt_idx[i]] for i in range(self.N)]

        self.data['P_opt'] = P_opt
        self.data['Q_opt'] = Q_opt
        self.data['x_opt'] = x_opt
        return {
            'message': str(result.solver.termination_condition),
            'status': str(result.solver.status),
            'model': self.model,
            'data': self.data,
            'x_sol': x_sol,
            'opt_idx': x_opt_idx
        }

</code></pre><p></p></div></details>

<details class="spoiler">
<summary>
|> Описание класа для MILP постановки чеерз Cvxpy
</summary>
<div class="spoiler__content"><pre><code class="python">
class CvxpyLpOptimizationModel:
    """
    Класс, который создаёт оптимизационную модель на базе библиотеки pyomo
    """
    def __init__(self, data: pd.DataFrame, alpha: int):
        if (alpha < 0.0) | (alpha > 1.0):
            raise ValueError('alpha должен быть между 0 и 1')
        self.alpha = float(alpha)
        self.data = data['data_milp'].copy()
        self.N = len(self.data['plu_line'])
        self.grid_size = self.data['grid_size'].values
        self.g_max = max(self.grid_size)
        self.plu_line = self.data['plu_line'].values
        self.n_plu = self.data['n_plu'].values
        self.P = self.data['P'].values
        self.P_idx = self.data['P_idx'].values
        self.PC = self.data['PC'].values
        self.Ps = np.array(self.data['Ps'].to_list())
        self.Qs = np.array(self.data['Qs'].to_list())
        self.C = self.data['C'].values.reshape(-1, 1)
        # границы для индексов
        self.xs = np.array(self.data['xs'].to_list())
        self.fixed = self.data['fixed'].values
        # Задаём объекты для формирования
        self.x = None
        self.obj = None
        self.constraints = []
        self.x_mask = None

    def init_variables(self):
        """
        Инициализация переменных в модели
        """
        self.x = cp.Variable(shape=(self.N, self.g_max), boolean=True)
        # должна быть хотя бы одна цена из диапазона
        # вспомогательная маска для упрощения матричных операций при формирований задачи
        mask_idx = np.repeat(np.arange(self.g_max), self.N).reshape(self.g_max, self.N).T
        mask = np.ones((self.N, self.g_max))
        mask[mask_idx > np.array(self.grid_size).reshape(-1, 1) - 1] = 0
        self.x_mask = mask
        con_any_price = cp.sum(cp.multiply(self.x, self.x_mask), axis=1) == 1
        self.constraints.append(con_any_price)

        if sum(self.fixed) > 0:
            con_fix = self.x[self.fixed == 1, self.P_idx[self.fixed == 1]] == 1
            self.constraints.append(con_fix)

    def init_objective(self):
        """
        Инициализация целевой функции(выручка или фронт-маржа)
        """
        self.obj = cp.Maximize(cp.sum(cp.multiply(self.x, (self.Ps - self.alpha * self.C) * self.Qs)))

    def add_con_rev(self, s_min, s_max=None):
        """
        Добавление в модель ограничения на РТО
        """
        con_mrg = cp.sum(cp.multiply(self.x, self.Ps * self.Qs)) >= s_min
        self.constraints.append(con_mrg)

    def add_con_mrg(self, m_min, m_max=None):
        """
        Добавление в модель ограничения на маржу
        """
        con_mrg = cp.sum(cp.multiply(self.x, (self.Ps - self.C) * self.Qs)) >= m_min
        self.constraints.append(con_mrg)

    def add_con_equal(self):
        pass

    def add_con_chg_cnt(self, nmax=10000):
        """
        Добавление ограничения на количество изменяемых цен
        """
        con_chg_cnt = cp.sum(
            cp.multiply(self.x[np.arange(self.N), self.P_idx], self.n_plu)[self.P_idx > 0]
        ) >= sum(self.n_plu) - nmax
        self.constraints.append(con_chg_cnt)

    def solve(self, solver='ECOS_BB', options={}):
        """
        Метод, запускающий решение поставленной оптимизационной задачи
        """
        problem = cp.Problem(self.obj, self.constraints)
        problem.solve(solver, **options)

        if self.x.value is None:
            return {
                'message': str(problem.solution.status),
                'status': str(problem.status),
                'model': problem,
                'data': self.data,
            }

        x_opt_idx = [np.argmax(self.x.value[i, :self.grid_size[i]]) for i in range(self.N)]
        x_opt = self.xs[np.arange(self.N), x_opt_idx]
        P_opt = self.Ps[np.arange(self.N), x_opt_idx]
        Q_opt = self.Qs[np.arange(self.N), x_opt_idx]
        self.data['P_opt'] = P_opt
        self.data['Q_opt'] = Q_opt
        self.data['x_opt'] = x_opt

        return {
            'message': str(problem.solution.status),
            'status': str(problem.status),
            'model': problem,
            'data': self.data,
            'opt_idx': x_opt_idx
        }
</code></pre><p></p></div></details>

Теперь можно считать, что все готово для запуска и тестирования оптимизаторов.
Рассмотрим на небольшом примере, что все отрабатывает корректно и выполняются все условия на примере двух классов оптимизационных моделей.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from data_generator.data_generator import generate_data, price_round
from optimizers.optimization import pricing_optimization
from optimizers.optimizers import (
    ScipyNlpOptimizationModel,
    PyomoNlpOptimizationModel,
    PyomoLpOptimizationModel,
    CvxpyLpOptimizationModel,
)

def calc_metrics(df, tp='cur'):
    sfx = ''
    if tp == 'cur':
        sfx = ''
    elif tp == 'opt':
        sfx = '_opt'
    R_ = sum(df['P' + sfx] * df['Q' + sfx])
    M_ = sum((df['P' + sfx] - df['C']) * df['Q' + sfx])
    return R_, M_

def perc_delta(v_old, v_new, ndigits=2):
    p = round(100. * (v_new / v_old - 1.), ndigits)
    sign = '+' if p >= 0 else '-'
    return sign + str(abs(p)) + '%'

bounds_params = {
    'main_bounds': {
        'lower': 0.9, 'upper': 1.1
    },
    'market_bounds': {
        'lower': 0.85, 'upper': 1.15
    }
}
grid_size = 21
# 
data = generate_data(50, bounds_params, grid_size, 0)
# текущая выручка и маржа
R_cur, M_cur = calc_metrics(data['data_nlp'], 'cur')
# параметры для оптимизации
opt_params = {
    'alpha': 0.0,
    'con_mrg': M_cur,
}

res_nlp, _ = pricing_optimization(data, PyomoNlpOptimizationModel, opt_params, 'ipopt')
res_milp, _ = pricing_optimization(data, CvxpyLpOptimizationModel, opt_params, 'GLPK_MI')

R_opt_nlp, M_opt_nlp = calc_metrics(res_nlp['data'], 'opt')
R_opt_milp, M_opt_milp = calc_metrics(res_milp['data'], 'opt')

print('dR = {dR}, dM = {dM} - Изменение выручки и маржи в NLP задаче'.format(
    dR=perc_delta(R_cur, R_opt_nlp), dM=perc_delta(M_cur, M_opt_nlp)))
print('dR = {dR}, dM = {dM} - Изменение выручки и маржи в MILP задаче'.format(
    dR=perc_delta(R_cur, R_opt_milp),
    dM=perc_delta(M_cur, M_opt_milp)))

res_nlp['data']['P_opt'] = price_round(res_nlp['data']['P_opt'])
res_nlp['data']['x_opt'] = res_nlp['data']['P_opt'] / res_nlp['data']['P']
res_nlp['data']['Q_opt'] = res_nlp['data']['Q'] * np.exp(res_nlp['data']['E'] * (res_nlp['data']['x_opt'] - 1))
R_opt_nlp_ar, M_opt_nlp_ar = calc_metrics(res_nlp['data'], 'opt')

print('dR = {dR}, dM = {dM} - Изменение выручки и маржи в NLP задаче после округления'.format(
    dR=perc_delta(R_cur, R_opt_nlp_ar), dM=perc_delta(M_cur, M_opt_nlp_ar)))

ind = res_nlp['data'].groupby(['plu_line_idx'])['x_opt'].nunique().max() == 1
print(f'Все товары в линейке имеют одинаковые цены: {ind}')


dR = +7.9%, dM = +0.0% - Изменени выручки и маржи в NLP задаче
dR = +7.87%, dM = +0.01% - Изменени выручки и маржи в MILP задаче
dR = +7.86%, dM = +0.62% - Изменени выручки и маржи в NLP задаче после округления
Все товары в линейке имют одинаковые цены: True


## Результаты расчётов

Посмотрим за какое время и как они справятся с разным количеством товаров $n$, читай размерностью задачи.
Для этого был написан [скрипт](https://github.com/mbudylin/OptimizersArticle/blob/main/runner.py), который пробегается по значениям $n$ = \[10, 20, 50, 100, 200, 500, 1000\] и данных,
сгенерированных для набора seed'ов от 0 до 24, итого 25 тестов для каждой модели и размерности.
Будем замерять время оптимизации, обернув функцию pricing_optimization декоратором.
Во время предварительных запусков стало понятно, что некоторые оптимизаторы плохо справляются с размерностями $\gtrsim$ 100, поэтому чтобы лишний раз не греть атмосферу, было решено ограничить $n$ сверху для них.

<details class="spoiler">
<summary>
|> Декоратор для замера времени
</summary>
<div class="spoiler__content"><pre><code class="python">
from time import time
from functools import wraps


def timeit(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        t0 = time()
        result = function(*args, **kwargs)
        duration = time() - t0
        return result, duration
    return wrapper
    
@timeit
def pricing_optimization(...):
    ....
</code></pre><p></p></div></details>

<details class="spoiler">
<summary>
|> Скрипт для замера времени
</summary>
<div class="spoiler__content"><pre><code class="python">
import os
import sys
from itertools import product
import pandas as pd

sys.path.append('./OptimizersArticle')

from data_generator.data_generator import generate_data

from optimizers.optimizers import (
    ScipyNlpOptimizationModel,
    PyomoNlpOptimizationModel,
    PyomoLpOptimizationModel,
    CvxpyLpOptimizationModel
)
from optimizers.optimization import pricing_optimization

STAT_PATH = './data/stat'

SEED_GRID = list(range(25))
N_GRID = [10, 20, 50, 100, 200, 500, 1000]
GRID = product(N_GRID, SEED_GRID)
BOUNDS_PARAMS = {
    'main_bounds': {
        'lower': 0.9, 'upper': 1.1
    },
    'market_bounds': {
        'lower': 0.85, 'upper': 1.15
    }
}
LP_MAX_GRID_SIZE = 21


def get_files(file_path):
    files = []
    if os.path.exists(file_path):
        files = [f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f)) and f.endswith('.csv')]
    return files


def optimizers_calc_stat(grid, file_path_stat, overwrite=False):

    existed_files = set(get_files(file_path_stat))

    for N, seed in grid:
        file_name = 's_' + str(N) + '_' + str(seed) + '.csv'

        # print(dump_key)
        # print(existed_stat)
        # assert ()
        if not overwrite:
            if file_name in existed_files:
                continue
        print('--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--')
        print(N, seed)
        # генерация данных для NLP и LP оптимизации
        data = generate_data(N, BOUNDS_PARAMS, LP_MAX_GRID_SIZE, seed)

        M_cur = sum(data['data_nlp']['Q'] * (data['data_nlp']['P'] - data['data_nlp']['C']))
        R_cur = sum(data['data_nlp']['Q'] * data['data_nlp']['P'])

        opt_params = {
            'alpha': 0.0,
            'con_mrg': M_cur,
            'con_equal': True
        }

        statuses = []
        times = []
        solvers = []

        if N < 1000:
            res, t = pricing_optimization(data, ScipyNlpOptimizationModel, opt_params, 'slsqp')
            statuses.append('ok' if res['status'] == '0' else res['status'])
            times.append(t)
            solvers.append('slsqp')
            print(f'slsqp finish \t{t}')

        if N < 500:
            res, t = pricing_optimization(data, ScipyNlpOptimizationModel, opt_params, 'trust-constr')
            statuses.append('ok' if res['status'] == '1' else res['status'])
            times.append(t)
            solvers.append('trust-constr')
            print(f'trust finish \t{t}')

        res, t = pricing_optimization(data, PyomoNlpOptimizationModel, opt_params, 'ipopt')
        statuses.append('ok' if res['status'] == 'ok' else res['status'])
        times.append(t)
        solvers.append('ipopt')
        print(f'ipopt finish \t{t}')

        res, t = pricing_optimization(data, PyomoLpOptimizationModel, opt_params, 'cbc')
        statuses.append('ok' if res['status'] == 'ok' else res['status'])
        times.append(t)
        solvers.append('pyomo_cbc')
        print(f'pyomo cbc finish \t{t}')

        res, t = pricing_optimization(data, PyomoLpOptimizationModel, opt_params, 'glpk')
        statuses.append('ok' if res['status'] == 'ok' else res['status'])
        times.append(t)
        solvers.append('pyomo_glpk')
        print(f'pyomo glpk finish \t{t}')

        res, t = pricing_optimization(data, CvxpyLpOptimizationModel, opt_params, 'CBC')
        statuses.append('ok' if res['status'] == 'optimal' else res['status'])
        times.append(t)
        solvers.append('cvxpy_cbc')
        print(f'cvxpy cbc finish \t{t}')

        res, t = pricing_optimization(data, CvxpyLpOptimizationModel, opt_params, 'GLPK_MI')
        statuses.append('ok' if res['status'] == 'optimal' else res['status'])
        times.append(t)
        solvers.append('cvxpy_glpk')
        print(f'cvxpy glpk finish \t{t}')

        if N < 2000:
            res, t = pricing_optimization(data, CvxpyLpOptimizationModel, opt_params, 'ECOS_BB')
            statuses.append('ok' if res['status'] == 'optimal' else res['status'])
            times.append(t)
            solvers.append('ecos')
            print(f'ecos finish \t{t}')

        df = pd.DataFrame({
            'N': N,
            'seed': seed,
            'solver': solvers,
            't': times,
            'status': statuses
        })
        print(os.path.join(file_path_stat, file_name))
        df.to_csv(os.path.join(file_path_stat, file_name), index=False)


def optimizers_collect_stat(file_path_stat):
    files_stat = get_files(file_path_stat)
    df = pd.concat([pd.read_csv(os.path.join(file_path_stat, file_name)) for file_name in files_stat])
    return df


if __name__ == '__main__':

    optimizers_calc_stat(GRID, STAT_PATH, overwrite=True)
    stat_df = optimizers_collect_stat(STAT_PATH)

</code></pre><p></p></div></details>


<img src="./images/time_solve_compare.png" width="1000" align="c"/>

Результаты замеров представлены выше на графиках, жирной линией отображена медиана, а полупрозрачный коридор — это нижняя и верхняя квартили.
По задаче NLP видно, что на низкоразмерных задачах (<100), slsqp(scipy) вполне себе является лидером, а вот при больших размерностях, явно побеждает ipopt(pyomo).
Trust-constr уже на старте всем уступает.
Для MILP задачи асимптотики у всех плюс-минус похожи, интересно что для одного и того же солвера пакеты pyomo и cvxpy дают результаты отличающиеся по времени работы.
А все потому что pyomo во время объявления переменных, целевой функции и ограничений часть времени тратит н на преобразование в вид, который позволит передать задачу дальше во внешний солвер.
Это может сильно повлиять на общее время решения, особенно это заметно, когда объявляется большое число ограничений.

### Учет дополнительных ограничений

Выше мы описали, что есть такая задача MINLP - это та в которой появляются целочисленные значения.
Как они могут возникнуть в ценообразовании? Если мы захотим каким-то образом управлять количеством цен к изменению, то непрерывной оптимизацией (по крайней мере честной) не обойдешься,
нужно включать дискретные правила.
Допустим мы не хотим менять цену, если она изменилась менее чем на 2%, тогда возникает "дырка" - область поиска становится разрывной.
Такой разрыв можно устранить/описать добавив индикатор того, где происходит поиск новой цены — либо слева, либо справа, либо в центре.
Тогда количеством изменяемых цен можно управлять через сумму индикаторов "в центре" - не меньше чем заданное число.

Здесь могла быть ваша реклама, но будет постановка ограничения на количество изменяемых цен в постановке MINLP

Задачу MINLP может решать bonmin в pyomo, дополним вызов оптимизатора добавлением ограничения на количество изменяемых цен сверху add_con_chg_cnt, которое опишем в классе PyomoNlpOptimizationModel.

<details class="spoiler">
<summary>
|> Метод add_con_chg_cnt и дополнительный ключ в параметрах
</summary>
<div class="spoiler__content"><pre><code class="python">
class PyomoNlpOptimizationModel
...   
    def add_con_chg_cnt(self, thr_l=0.98, thr_u=1.02, nmax=10000):
        """
        Добавление в модель ограничения на изменение цен
        """
        self.model.ind_l = pyo.Var(range(self.N), domain=pyo.Binary, initialize=0)
        self.model.ind_r = pyo.Var(range(self.N), domain=pyo.Binary, initialize=0)
        self.model.ind_m = pyo.Var(range(self.N), domain=pyo.Binary, initialize=1)

        self.model.x_interval = pyo.Constraint(pyo.Any)
        self.model.con_ind = pyo.Constraint(pyo.Any)

        for i in range(self.N):
            self.model.x_interval['l' + str(i)] = self.model.x[i] <= thr_l + 10. * (1 - self.model.ind_l[i])
            self.model.x_interval['r' + str(i)] = self.model.x[i] >= thr_u - 10. * (1 - self.model.ind_r[i])
            self.model.x_interval['ml' + str(i)] = self.model.x[i] <= 1. + 10. * (1 - self.model.ind_m[i])
            self.model.x_interval['mr' + str(i)] = self.model.x[i] >= 1. - 10. * (1 - self.model.ind_m[i])
            self.model.con_ind[i] = (self.model.ind_l[i] + self.model.ind_m[i] + self.model.ind_r[i]) == 1

        self.model.con_max_chg = pyo.Constraint(expr=sum(
            self.model.ind_m[i]
            for i in range(self.N)
        ) >= self.N - nmax)

opt_params = {
    ...
    'add_con_chg_cnt': N_chg_max
}


def pricing_optimization(...):
    ...(после инициализации переменных)
    
    if opt_params.get('con_chg_cnt') is not None:
        model.add_con_chg_cnt(opt_params['con_chg_cnt'])
   ...

</code></pre><p></p></div></details>


In [31]:
import numpy as np
import matplotlib.pyplot as plt
from data_generator.data_generator import generate_data
from optimizers.optimization import pricing_optimization
from optimizers.optimizers import PyomoNlpOptimizationModel

bounds_params = {
    'main_bounds': {
        'lower': 0.9, 'upper': 1.1
    },
    'market_bounds': {
        'lower': 0.85, 'upper': 1.15
    }
}
grid_size = 21
N_plu_line = 100
data = generate_data(20, bounds_params, grid_size, 0)
N_plu = len(data['data_nlp'])
# текущая выручка и маржа
R_cur, M_cur = calc_metrics(data['data_nlp'], 'cur')
# параметры для оптимизации
opt_params = {
    'alpha': 0.0,
    'con_mrg': M_cur,
    'con_chg_cnt':  int(0.7 * N_plu)
}
res_minlp, t_minlp = pricing_optimization(data, PyomoNlpOptimizationModel, opt_params, 'bonmin')
print('%.3f' % t_minlp)

5.787


Как можно видеть, время решения задачи серьезно увеличивается, что не удивительно, саму по себе задачу NLP не всегда просто решить,
а тут еще нужно учесть дискретные правила, которые увеличивают сложность, куда гармоничнее эти правила укладываются в MILP постановку,
так как такие задачи достаточно хорошо научились решать, что нельзя сказать про MINLP.
Дополним задачу (3) ограничением на общее количество изменений цены, обозначим это число $d$.
Его проще всего сформулировать через количество неизменяемых цен, для этого нужно иметь индекс текущей цены в сетке $p_{i}$ для $i$-ой линейки,
если, конечно, он там есть, обозначим это индикатором $I[p_{i} \in g_{i}]$ - равен единичке, если текущая цена в сетке и ноль противном случае

\begin{equation}
\tag{3.5}
\sum_{i=1}^{n'} I |\gamma_{i}| \hat{x}_{i, p_{i}} \geqslant n - d
\end{equation}

Здесь $|\gamma_{i}|$ - размер $i$-ой линейки

In [32]:
opt_params = {
    'alpha': 0.0,
    'con_mrg': M_cur,
    'con_chg_cnt':  int(0.7 * N_plu)
}
res_milp, t_milp = pricing_optimization(data, CvxpyLpOptimizationModel, opt_params, 'GLPK_MI')
res_df = res_milp['data']
print('%.3f' % t_milp)

0.036


In [33]:
# доля изменяемых цен
p_chg = res_df[(res_df['x_opt'] - 1).abs() > 1.0e-6]['n_plu'].sum() / res_df['n_plu'].sum()
print('%.3f' % p_chg)

0.690
